![alt text](track.jpg "Title")

<div>
<img src="small_track.png" width="400"/>
</div>


In [ ]:
%%clingo 0 environment.lp track_option.lp -

option((X,Y),TO) :- track_option(Track,TO),  cell((X,Y), Track).

% at time 0 trains should be found at their starting position
at(train(ID),(X,Y),Dep,Dir) :- time(Dep), start(ID,(X,Y),Dep,Dir).

% add wait actions before and after trains' start and end times
action(train(ID),wait,T) :- T < ST, start(ID,_,ST,_), time(T).
action(train(ID),wait,T) :- T > ET, end(ID,_,ET), time(T).

% TODO: should actions be happening at T or T+1? At T=0 no action?
% add actions based on train position and cell options
{ action(train(ID),M,T+1): option((X,Y),(D,M,_)) } = 1 :-
    at(train(ID),(X,Y),T,D), T < ET, end(ID,_,ET), time(T+1).

% only one action per train and time T
:- M1 != M2, action(train(ID),M1,T), action(train(ID),M2,T).

% only one at movement per train and time T
:- D1 != D2, at(train(ID),(_,_),T,D1), at(train(ID),(_,_),T,D2).
:- ID1 != ID2, at(train(ID1),(X,Y),T,_), at(train(ID2),(X,Y),T,_).

% ensure trains reach their destinations
:- not at(train(ID),(X,Y),_,_), end(ID,(X,Y),_).

% if moving forward add ats to indicate new cell positions
at(train(ID),(X,Y),T2,D) :- option((X,Y),(D,wait,D)), action(train(ID),wait,T2),
    at(train(ID),(X,Y),T1,D), T2 = T1 + 1, time(T2), time(T1).

at(train(ID),(X2,Y),T2,n) :- X2 = X1-1, option((X1,Y),(D,M,n)),
    cell((X2,Y),_), action(train(ID),M,T2), at(train(ID),(X1,Y),T1,D),
    T2 = T1 + 1, time(T2), time(T1).

at(train(ID),(X,Y2),T2,e) :- Y2 = Y1+1, option((X,Y1),(D,M,e)),
    cell((X,Y2),_), action(train(ID),M,T2), at(train(ID),(X,Y1),T1,D),
    T2 = T1 + 1, time(T2), time(T1).

at(train(ID),(X2,Y),T2,s) :- X2 = X1+1, option((X1,Y),(D,M,s)),
    cell((X2,Y),_), action(train(ID),M,T2), at(train(ID),(X1,Y),T1,D),
    T2 = T1 + 1, time(T2), time(T1).

at(train(ID),(X,Y2),T2,w) :- Y2 = Y1-1, option((X,Y1),(D,M,w)),
    cell((X,Y2),_), action(train(ID),M,T2), at(train(ID),(X,Y1),T1,D),
    T2 = T1 + 1, time(T2), time(T1).

% TODO: max-/minimize statement could be based on num of waits within each
%     trains' accepted movement timeframe?
% #maximize{ 1,T,M,N: at(train(ID),(X,Y2),T2,w) }.

% #show option/2.
#show at/4.
#show action/3.





In [ ]:


#script(python)
from clingo.symbol import Tuple_, Number

def options(T, move_left, move_forward, move_right, wait, n, e, s, w):
    t = T.number
    t_bit = '{0:016b}'.format(t)
    
    options = []
    
    #######################################################
    directions = {
        0: n,
        1: e,
        2: s,
        3: w,
    }
    
    cell = {}
    for i in range(0, 4):
        in_dir = directions[i]
        section = list(t_bit[4*i:4*(i+1)])
        if section.count('1') == 0:
            continue
        if section.count('1') == 1:
            options.append(
                Tuple_([in_dir,
                        move_forward,
                        directions[section.index('1')]]))
        else:
            moves = {
                i - 1 if i != 0 else 3: move_left,
                i: move_forward,
                i + 1 if i != 3 else 0: move_right,
            }
            for j, subsection in enumerate(section):
                if subsection == '1':
                    options.append(Tuple_([in_dir,
                                           moves[j],
                                           directions[j]]))
            # options.append(Tuple_([in_dir, wait, in_dir]))

    #######################################################
    # for i in range(0, 4):
    #     section = t_bit[4*i:4*(i+1)]
    #     if section != '0000':
    #         move = [directions[i]]
    #         for j in range(0, 4):
    #             move.append(Number(int(section[j])))
    #         options.append(Tuple_(move))
    #######################################################

    return options
#end.

% option((X,Y),@options(Track,move_left,move_forward,move_right,wait,n,e,s,w))
%     :- cell((X,Y), Track).

% TODO: is the following code faster than the above code?
% track_option(Track,
%               @options(Track,move_left,move_forward,move_right,wait,n,e,s,w))
%     :- cell(_, Track).
option((X,Y),TO) :- track_option(Track,TO),  cell((X,Y), Track).

# Notes

cell: `cell((X,Y), Track)`\
train: `train(ID)`\
    start: `start(ID,(X,Y),Dep,Dir)`\
    end: `end(ID,(X,Y),Arr)`\
action:
`action(train(ID), move_left, T)`\
`action(train(ID), move_right, T)`\
`action(train(ID), wait, T)`\
`action(train(ID), move_forward, T)`\


- The track types have been added to this section as a new reference file
- In the start(ID,(X,Y),Dep,Dir) fact, the Dir variable is the direction it faces {n,e,s,w}
- In the start(ID,(X,Y),Dep,Dir) fact, the Dep variable is the earliest departure time
- In the end(ID,(X,Y),Arr) fact, the Arr variable is the latest arrival time
- Therefore, Dep - Arr forms a time window during which the train is permitted to move
- The ending direction of the train is not important—what matters is that it reaches its destination


From the presentation:
- don't need to include the option to wait at every cell, more efficient to skip over them
- never in a situation where a train would have the option to go left, forward, and right. if a train can go forward and left, the forward could also be considered going right, this distinction is up to us
- no deadends, no turning around, also no sharing a cell even in the case of two curved tracks sharing a cell but not touching each other (which were also removed)
- in flatlands the little pink/red building (train station?) indicate where the trains will end, from looking at a png of a geenerated track there is no clear way to know where the trains start
- ALSO axes are switched: top horizontal axis is Y, left vertical is X, top left is (0,0), bottom right is (n,n)
- first action actually spawns the train onto the map, need to add an action for the train to be added.  So time T = 1 the train will be at the starting position?

